In [3]:
import os
import sys
sys.path.insert(0,'..')

from bidst.transformers import NUCorrectionTransformer
from bidst.transformers import SkullStrippingTransformer
from bidst.transformers import TissueSegmentation
from bidst.transformers import StatAggregator
from bidst.pipeline import BIDSPipeline

from sklearn.linear_model import LogisticRegression
import numpy as np

In [4]:
import nibabel as nib
from functools import reduce
from operator import mul

def get_tissue_stats(subject_folder):
    
    ext_dict = {}
    ext_dict['WM_volume'] = 'WM_probtissue'
    ext_dict['GM_volume'] = 'GM_probtissue'
    ext_dict['CSF_volume'] = 'CSF_probtissue'
    
    files_list = os.listdir(subject_folder)
    
    results = {}
    for key in ext_dict.keys():
        cur_file = [item for item in files_list if ext_dict[key] in item]
        if len(cur_file) > 1:
            raise ValueError('Several candiadates in the folder')
        else:
            cur_file = cur_file[0]
        cur_img = nib.load(subject_folder + '/' + cur_file)
        voxel_sum = cur_img.get_data()
        voxel_sum = (voxel_sum >= 0.5).sum()
        voxel_vol = reduce(mul, cur_img.header.get_zooms())
        all_vol = voxel_sum * voxel_vol
        results[key] = all_vol
    
    return results

In [3]:
!tree -a ../bidst/tests/data/ds114/

../bidst/tests/data/ds114/
├── derivatives
│   ├── test_pipeline
│   │   └── steps
│   │       ├── skullstripping
│   │       │   └── sub-03
│   │       │       ├── ses-retest
│   │       │       │   └── anat
│   │       │       │       └── sub-03_ses-retest_T1w_brain.nii.gz
│   │       │       └── ses-test
│   │       │           └── anat
│   │       │               └── sub-03_ses-test_T1w_brain.nii.gz
│   │       └── tissuesegmentation
│   │           └── sub-03
│   │               ├── ses-retest
│   │               │   └── anat
│   │               │       ├── sub-03_ses-retest_T1w_brain_class-CSF_probtissue.nii.gz
│   │               │       ├── sub-03_ses-retest_T1w_brain_class-GM_probtissue.nii.gz
│   │               │       ├── sub-03_ses-retest_T1w_brain_class-WM_probtissue.nii.gz
│   │               │       ├── sub-03_ses-retest_T1w_brain_mixeltype.nii.gz
│   │               │       ├── sub-03_ses-retest_T1w_brain_pveseg.nii.gz
│   │               │       └── sub-03_ses-retest_

In [ ]:
project_path = os.path.abspath('..') + '/bidst/tests/data/ds114'
pipeline_name = 'test_pipeline0'

nucorrection = NUCorrectionTransformer(gather_steps=('source', dict(extensions='T1w.nii.gz')))

skullstripping = SkullStrippingTransformer(gather_steps=('nucorrection', dict(extensions='nu.nii.gz')))

tissuesegmentation = TissueSegmentation(gather_steps=('skullstripping', dict(extensions='brain.nii.gz')))

aggregator = StatAggregator(function=get_tissue_stats, 
                            gather_steps=('tissuesegmentation', dict(extensions='probtissue.nii.gz')))

IDS = ['01']

pipe = BIDSPipeline(steps=[('nucorrection', nucorrection),
                           ('skullstripping', skullstripping),
                           ('tissuesegmentation', tissuesegmentation),
                           ('aggregator', aggregator)],
             pipeline_name=pipeline_name, project_path=project_path)

In [ ]:
pipe.fit_transform(IDS)

/BIDST/bidst/tests/data/ds114/derivatives/test_pipeline0/steps/nucorrection/sub-01/ses-retest/anat/sub-01_ses-retest_T1w_nu.nii.gz
170831-21:01:32,209 interface INFO:
	 stdout 2017-08-31T21:01:32.208919:
170831-21:01:32,210 interface INFO:
	 stdout 2017-08-31T21:01:32.208919:Running N4 for 3-dimensional images.
170831-21:01:32,211 interface INFO:
	 stdout 2017-08-31T21:01:32.208919:
170831-21:01:32,212 interface INFO:
	 stdout 2017-08-31T21:01:32.208919:Mask not read.  Creating Otsu mask.
170831-21:01:33,224 interface INFO:
	 stdout 2017-08-31T21:01:33.224815:Current level = 1
170831-21:01:33,226 interface INFO:
	 stdout 2017-08-31T21:01:33.224815:  Iteration 1 (of 50).   Current convergence value = 0.000264527 (threshold = 0)
170831-21:01:33,227 interface INFO:
	 stdout 2017-08-31T21:01:33.224815:  Iteration 2 (of 50).   Current convergence value = 0.000262299 (threshold = 0)
170831-21:01:33,228 interface INFO:
	 stdout 2017-08-31T21:01:33.224815:  Iteration 3 (of 50).   Current conve

170831-21:01:40,812 interface INFO:
	 stdout 2017-08-31T21:01:40.812054:Current level = 2
170831-21:01:40,813 interface INFO:
	 stdout 2017-08-31T21:01:40.812054:  Iteration 1 (of 50).   Current convergence value = 0.000425829 (threshold = 0)
170831-21:01:40,814 interface INFO:
	 stdout 2017-08-31T21:01:40.812054:  Iteration 2 (of 50).   Current convergence value = 0.000419074 (threshold = 0)
170831-21:01:40,815 interface INFO:
	 stdout 2017-08-31T21:01:40.812054:  Iteration 3 (of 50).   Current convergence value = 0.000411931 (threshold = 0)
170831-21:01:40,816 interface INFO:
	 stdout 2017-08-31T21:01:40.812054:  Iteration 4 (of 50).   Current convergence value = 0.000406177 (threshold = 0)
170831-21:01:41,318 interface INFO:
	 stdout 2017-08-31T21:01:41.318658:  Iteration 5 (of 50).   Current convergence value = 0.000400012 (threshold = 0)
170831-21:01:41,319 interface INFO:
	 stdout 2017-08-31T21:01:41.318658:  Iteration 6 (of 50).   Current convergence value = 0.000393745 (thresho

170831-21:01:48,387 interface INFO:
	 stdout 2017-08-31T21:01:48.387613:  Iteration 4 (of 50).   Current convergence value = 0.000926239 (threshold = 0)
170831-21:01:48,389 interface INFO:
	 stdout 2017-08-31T21:01:48.387613:  Iteration 5 (of 50).   Current convergence value = 0.000895458 (threshold = 0)
170831-21:01:48,390 interface INFO:
	 stdout 2017-08-31T21:01:48.387613:  Iteration 6 (of 50).   Current convergence value = 0.000869874 (threshold = 0)
170831-21:01:48,391 interface INFO:
	 stdout 2017-08-31T21:01:48.387613:  Iteration 7 (of 50).   Current convergence value = 0.000843455 (threshold = 0)
170831-21:01:48,893 interface INFO:
	 stdout 2017-08-31T21:01:48.893633:  Iteration 8 (of 50).   Current convergence value = 0.000818881 (threshold = 0)
170831-21:01:48,894 interface INFO:
	 stdout 2017-08-31T21:01:48.893633:  Iteration 9 (of 50).   Current convergence value = 0.000789733 (threshold = 0)
170831-21:01:48,895 interface INFO:
	 stdout 2017-08-31T21:01:48.893633:  Iteratio

170831-21:01:55,460 interface INFO:
	 stdout 2017-08-31T21:01:55.458982:  Iteration 7 (of 50).   Current convergence value = 0.00111118 (threshold = 0)
170831-21:01:55,461 interface INFO:
	 stdout 2017-08-31T21:01:55.458982:  Iteration 8 (of 50).   Current convergence value = 0.0010646 (threshold = 0)
170831-21:01:55,962 interface INFO:
	 stdout 2017-08-31T21:01:55.962659:  Iteration 9 (of 50).   Current convergence value = 0.00102261 (threshold = 0)
170831-21:01:55,963 interface INFO:
	 stdout 2017-08-31T21:01:55.962659:  Iteration 10 (of 50).   Current convergence value = 0.000982827 (threshold = 0)
170831-21:01:55,964 interface INFO:
	 stdout 2017-08-31T21:01:55.962659:  Iteration 11 (of 50).   Current convergence value = 0.000946726 (threshold = 0)
170831-21:01:55,966 interface INFO:
	 stdout 2017-08-31T21:01:55.962659:  Iteration 12 (of 50).   Current convergence value = 0.000914036 (threshold = 0)
170831-21:01:56,468 interface INFO:
	 stdout 2017-08-31T21:01:56.468398:  Iteration

170831-21:02:01,36 interface INFO:
	 stdout 2017-08-31T21:02:01.014186:     Required Input Names: Primary
170831-21:02:01,37 interface INFO:
	 stdout 2017-08-31T21:02:01.014186:     NumberOfRequiredInputs: 1
170831-21:02:01,39 interface INFO:
	 stdout 2017-08-31T21:02:01.014186:     Outputs: 
170831-21:02:01,40 interface INFO:
	 stdout 2017-08-31T21:02:01.014186:       Primary: (0x1e05f50)
170831-21:02:01,43 interface INFO:
	 stdout 2017-08-31T21:02:01.014186:     Indexed Outputs: 
170831-21:02:01,44 interface INFO:
	 stdout 2017-08-31T21:02:01.014186:       0: Primary (0x1e05f50)
170831-21:02:01,46 interface INFO:
	 stdout 2017-08-31T21:02:01.014186:     NumberOfRequiredOutputs: 1
170831-21:02:01,47 interface INFO:
	 stdout 2017-08-31T21:02:01.014186:     Number Of Threads: 1
170831-21:02:01,49 interface INFO:
	 stdout 2017-08-31T21:02:01.014186:     ReleaseDataFlag: Off
170831-21:02:01,50 interface INFO:
	 stdout 2017-08-31T21:02:01.014186:     ReleaseDataBeforeUpdateFlag: Off
170831

...

170831-21:02:11,873 interface INFO:
	 stdout 2017-08-31T21:02:11.873102:  Iteration 38 (of 50).   Current convergence value = 0.000228923 (threshold = 0)
170831-21:02:11,874 interface INFO:
	 stdout 2017-08-31T21:02:11.873102:  Iteration 39 (of 50).   Current convergence value = 0.000227733 (threshold = 0)
170831-21:02:11,875 interface INFO:
	 stdout 2017-08-31T21:02:11.873102:  Iteration 40 (of 50).   Current convergence value = 0.000226541 (threshold = 0)
170831-21:02:11,877 interface INFO:
	 stdout 2017-08-31T21:02:11.873102:  Iteration 41 (of 50).   Current convergence value = 0.000225292 (threshold = 0)
170831-21:02:12,379 interface INFO:
	 stdout 2017-08-31T21:02:12.378969:  Iteration 42 (of 50).   Current convergence value = 0.00022387 (threshold = 0)
170831-21:02:12,380 interface INFO:
	 stdout 2017-08-31T21:02:12.378969:  Iteration 43 (of 50).   Current convergence value = 0.000222706 (threshold = 0)
170831-21:02:12,381 interface INFO:
	 stdout 2017-08-31T21:02:12.378969:  Ite

170831-21:02:18,444 interface INFO:
	 stdout 2017-08-31T21:02:18.444474:  Iteration 41 (of 50).   Current convergence value = 0.000249309 (threshold = 0)
170831-21:02:18,445 interface INFO:
	 stdout 2017-08-31T21:02:18.444474:  Iteration 42 (of 50).   Current convergence value = 0.000246471 (threshold = 0)
170831-21:02:18,446 interface INFO:
	 stdout 2017-08-31T21:02:18.444474:  Iteration 43 (of 50).   Current convergence value = 0.000243748 (threshold = 0)
170831-21:02:18,447 interface INFO:
	 stdout 2017-08-31T21:02:18.444474:  Iteration 44 (of 50).   Current convergence value = 0.000240966 (threshold = 0)
170831-21:02:18,948 interface INFO:
	 stdout 2017-08-31T21:02:18.948671:  Iteration 45 (of 50).   Current convergence value = 0.000238042 (threshold = 0)
170831-21:02:18,949 interface INFO:
	 stdout 2017-08-31T21:02:18.948671:  Iteration 46 (of 50).   Current convergence value = 0.000235279 (threshold = 0)
170831-21:02:18,950 interface INFO:
	 stdout 2017-08-31T21:02:18.948671:  It

170831-21:02:25,17 interface INFO:
	 stdout 2017-08-31T21:02:25.016022:  Iteration 44 (of 50).   Current convergence value = 0.000465534 (threshold = 0)
170831-21:02:25,18 interface INFO:
	 stdout 2017-08-31T21:02:25.016022:  Iteration 45 (of 50).   Current convergence value = 0.000456484 (threshold = 0)
170831-21:02:25,19 interface INFO:
	 stdout 2017-08-31T21:02:25.016022:  Iteration 46 (of 50).   Current convergence value = 0.000450841 (threshold = 0)
170831-21:02:25,523 interface INFO:
	 stdout 2017-08-31T21:02:25.523051:  Iteration 47 (of 50).   Current convergence value = 0.000441746 (threshold = 0)
170831-21:02:25,524 interface INFO:
	 stdout 2017-08-31T21:02:25.523051:  Iteration 48 (of 50).   Current convergence value = 0.000434456 (threshold = 0)
170831-21:02:25,525 interface INFO:
	 stdout 2017-08-31T21:02:25.523051:  Iteration 49 (of 50).   Current convergence value = 0.000429248 (threshold = 0)
170831-21:02:26,27 interface INFO:
	 stdout 2017-08-31T21:02:26.027334:  Iterat

170831-21:02:32,599 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:  Iteration 47 (of 50).   Current convergence value = 0.000507059 (threshold = 0)
170831-21:02:32,600 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:  Iteration 48 (of 50).   Current convergence value = 0.000503925 (threshold = 0)
170831-21:02:32,602 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:  Iteration 49 (of 50).   Current convergence value = 0.000499756 (threshold = 0)
170831-21:02:32,604 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:  Iteration 50 (of 50).   Current convergence value = 0.000496092 (threshold = 0)
170831-21:02:32,605 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:   N4BiasFieldCorrectionImageFilter (0x232a320)
170831-21:02:32,606 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:     RTTI typeinfo:   itk::N4BiasFieldCorrectionImageFilter<itk::Image<float, 3u>, itk::Image<float, 3u>, itk::Image<float, 3u> >
170831-21:02:32,609 interface INFO:
	 stdout 2017-08-31T21:02:

170831-21:02:32,764 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:           Size: [11, 11, 11]
170831-21:02:32,765 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:         RequestedRegion: 
170831-21:02:32,767 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:           Dimension: 3
170831-21:02:32,769 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:           Index: [0, 0, 0]
170831-21:02:32,771 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:           Size: [11, 11, 11]
170831-21:02:32,772 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:         Spacing: [31.5, 24.6881, 31.5]
170831-21:02:32,774 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:         Origin: [30.8859, 22.742, -33.5092]
170831-21:02:32,775 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:         Direction: 
170831-21:02:32,777 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:-0.999132 0.0397339 -0.0125136
170831-21:02:32,778 interface INFO:
	 stdout 2017-08-31T21:02:32.599391:-0.0

In [ ]:
!tree -a ../bidst/tests/data/ds114/

In [ ]:
project_path = os.path.abspath('..') + '/bidst/tests/data/ds114'
pipeline_name = 'test_pipeline1'

skullstripping = SkullStrippingTransformer(gather_steps=('source', dict(extensions='T1w.nii.gz')))

tissuesegmentation = TissueSegmentation(gather_steps=('skullstripping', dict(extensions='T1w_brain.nii.gz')))

aggregator = StatAggregator(function=get_tissue_stats,
                            gather_steps=('tissuesegmentation', dict(extensions='WM_probtissue.nii.gz')))

IDS = ['02']


pipe = BIDSPipeline(steps=[('skullstripping', skullstripping),
                           ('tissuesegmentation', tissuesegmentation),
                           ('aggregator', aggregator)],
             pipeline_name=pipeline_name, project_path=project_path)

In [ ]:
pipe.fit_transform(IDS)

In [ ]:
!tree -a ../bidst/tests/data/ds114/

In [ ]:
project_path = os.path.abspath('..') + '/bidst/tests/data/ds114'
pipeline_name = 'test_pipeline2'

nucorrection = NUCorrectionTransformer(gather_steps=('source', dict(extensions='T1w.nii.gz')))

tissuesegmentation = TissueSegmentation(gather_steps=('nucorrection', dict(extensions='nu.nii.gz')))

aggregator = StatAggregator(function=get_tissue_stats,
                            gather_steps=('tissuesegmentation', dict(extensions='probtissue.nii.gz')))

IDS = ['03']

pipe = BIDSPipeline(steps=[('nucorrection', nucorrection),
                           ('tissuesegmentation', tissuesegmentation),
                           ('aggregator', aggregator)],
             pipeline_name=pipeline_name, project_path=project_path)

In [ ]:
pipe.fit_transform(IDS)

In [ ]:
!tree -a ../bidst/tests/data/ds114/

In [5]:
project_path = os.path.abspath('..') + '/bidst/tests/data/ds114'
pipeline_name = 'test_pipeline_with_ml'

nucorrection = NUCorrectionTransformer(gather_steps=('source', dict(extensions='T1w.nii.gz')))

skullstripping = SkullStrippingTransformer(gather_steps=('nucorrection', dict(extensions='nu.nii.gz')))

tissuesegmentation = TissueSegmentation(gather_steps=('skullstripping', dict(extensions='brain.nii.gz')))

aggregator = StatAggregator(function=get_tissue_stats, 
                            gather_steps=('tissuesegmentation', dict(extensions='probtissue.nii.gz')))

logisticregression = LogisticRegression()

IDS = ['01', '02']
target = np.array([0, 0, 1, 1])

pipe = BIDSPipeline(steps=[('nucorrection', nucorrection),
                           ('skullstripping', skullstripping),
                           ('tissuesegmentation', tissuesegmentation),
                           ('aggregator', aggregator),
                           ('logisticregression', logisticregression)],
             pipeline_name=pipeline_name, project_path=project_path)

In [ ]:
pipe.fit(X=IDS, y=target)

/work/soft/BIDSTransformers/bidst/tests/data/ds114/derivatives/test_pipeline_with_ml/steps/nucorrection/sub-01/ses-retest/anat/sub-01_ses-retest_T1w_nu.nii.gz
/work/soft/BIDSTransformers/bidst/tests/data/ds114/sub-01/ses-retest/anat/sub-01_ses-retest_T1w.nii.gz
170901-20:24:40,156 interface INFO:
	 stdout 2017-09-01T20:24:40.156778:
170901-20:24:40,158 interface INFO:
	 stdout 2017-09-01T20:24:40.156778:Running N4 for 3-dimensional images.
170901-20:24:40,159 interface INFO:
	 stdout 2017-09-01T20:24:40.156778:
170901-20:24:40,160 interface INFO:
	 stdout 2017-09-01T20:24:40.156778:Mask not read.  Creating Otsu mask.
170901-20:24:40,663 interface INFO:
	 stdout 2017-09-01T20:24:40.663355:Current level = 1
170901-20:24:40,664 interface INFO:
	 stdout 2017-09-01T20:24:40.663355:  Iteration 1 (of 50).   Current convergence value = 0.000264527 (threshold = 0)
170901-20:24:41,166 interface INFO:
	 stdout 2017-09-01T20:24:41.166460:  Iteration 2 (of 50).   Current convergence value = 0.00026

170901-20:24:48,230 interface INFO:
	 stdout 2017-09-01T20:24:48.230789:  Iteration 50 (of 50).   Current convergence value = 0.000193532 (threshold = 0)
170901-20:24:48,231 interface INFO:
	 stdout 2017-09-01T20:24:48.230789:Current level = 2
170901-20:24:48,232 interface INFO:
	 stdout 2017-09-01T20:24:48.230789:  Iteration 1 (of 50).   Current convergence value = 0.000425829 (threshold = 0)
170901-20:24:48,234 interface INFO:
	 stdout 2017-09-01T20:24:48.230789:  Iteration 2 (of 50).   Current convergence value = 0.000419074 (threshold = 0)
170901-20:24:48,235 interface INFO:
	 stdout 2017-09-01T20:24:48.230789:  Iteration 3 (of 50).   Current convergence value = 0.000411931 (threshold = 0)
170901-20:24:48,737 interface INFO:
	 stdout 2017-09-01T20:24:48.737200:  Iteration 4 (of 50).   Current convergence value = 0.000406177 (threshold = 0)
170901-20:24:48,738 interface INFO:
	 stdout 2017-09-01T20:24:48.737200:  Iteration 5 (of 50).   Current convergence value = 0.000400012 (thresh

170901-20:24:55,312 interface INFO:
	 stdout 2017-09-01T20:24:55.312872:  Iteration 3 (of 50).   Current convergence value = 0.000954957 (threshold = 0)
170901-20:24:55,313 interface INFO:
	 stdout 2017-09-01T20:24:55.312872:  Iteration 4 (of 50).   Current convergence value = 0.000926239 (threshold = 0)
170901-20:24:55,315 interface INFO:
	 stdout 2017-09-01T20:24:55.312872:  Iteration 5 (of 50).   Current convergence value = 0.000895458 (threshold = 0)
170901-20:24:55,316 interface INFO:
	 stdout 2017-09-01T20:24:55.312872:  Iteration 6 (of 50).   Current convergence value = 0.000869874 (threshold = 0)
170901-20:24:55,818 interface INFO:
	 stdout 2017-09-01T20:24:55.818449:  Iteration 7 (of 50).   Current convergence value = 0.000843455 (threshold = 0)
170901-20:24:55,819 interface INFO:
	 stdout 2017-09-01T20:24:55.818449:  Iteration 8 (of 50).   Current convergence value = 0.000818881 (threshold = 0)
170901-20:24:55,820 interface INFO:
	 stdout 2017-09-01T20:24:55.818449:  Iteratio

170901-20:25:01,928 interface INFO:
	 stdout 2017-09-01T20:25:01.926969:  Iteration 6 (of 50).   Current convergence value = 0.00116415 (threshold = 0)
170901-20:25:01,929 interface INFO:
	 stdout 2017-09-01T20:25:01.926969:  Iteration 7 (of 50).   Current convergence value = 0.00111118 (threshold = 0)
170901-20:25:02,431 interface INFO:
	 stdout 2017-09-01T20:25:02.431256:  Iteration 8 (of 50).   Current convergence value = 0.0010646 (threshold = 0)
170901-20:25:02,432 interface INFO:
	 stdout 2017-09-01T20:25:02.431256:  Iteration 9 (of 50).   Current convergence value = 0.00102261 (threshold = 0)
170901-20:25:02,433 interface INFO:
	 stdout 2017-09-01T20:25:02.431256:  Iteration 10 (of 50).   Current convergence value = 0.000982827 (threshold = 0)
170901-20:25:02,434 interface INFO:
	 stdout 2017-09-01T20:25:02.431256:  Iteration 11 (of 50).   Current convergence value = 0.000946726 (threshold = 0)
170901-20:25:02,936 interface INFO:
	 stdout 2017-09-01T20:25:02.936533:  Iteration 1

170901-20:25:08,1 interface INFO:
	 stdout 2017-09-01T20:25:07.982988:       1: _1 (0x20cfb70)
170901-20:25:08,2 interface INFO:
	 stdout 2017-09-01T20:25:07.982988:     Required Input Names: Primary
170901-20:25:08,3 interface INFO:
	 stdout 2017-09-01T20:25:07.982988:     NumberOfRequiredInputs: 1
170901-20:25:08,5 interface INFO:
	 stdout 2017-09-01T20:25:07.982988:     Outputs: 
170901-20:25:08,6 interface INFO:
	 stdout 2017-09-01T20:25:07.982988:       Primary: (0x20c4f90)
170901-20:25:08,7 interface INFO:
	 stdout 2017-09-01T20:25:07.982988:     Indexed Outputs: 
170901-20:25:08,9 interface INFO:
	 stdout 2017-09-01T20:25:07.982988:       0: Primary (0x20c4f90)
170901-20:25:08,10 interface INFO:
	 stdout 2017-09-01T20:25:07.982988:     NumberOfRequiredOutputs: 1
170901-20:25:08,11 interface INFO:
	 stdout 2017-09-01T20:25:07.982988:     Number Of Threads: 1
170901-20:25:08,13 interface INFO:
	 stdout 2017-09-01T20:25:07.982988:     ReleaseDataFlag: Off
170901-20:25:08,14 interfa

...

170901-20:26:28,31 interface INFO:
	 stdout 2017-09-01T20:26:28.031724:  Iteration 11 (of 50).   Current convergence value = 0.000240322 (threshold = 0)
170901-20:26:28,32 interface INFO:
	 stdout 2017-09-01T20:26:28.031724:  Iteration 12 (of 50).   Current convergence value = 0.00023624 (threshold = 0)
170901-20:26:28,34 interface INFO:
	 stdout 2017-09-01T20:26:28.031724:  Iteration 13 (of 50).   Current convergence value = 0.000231808 (threshold = 0)
170901-20:26:28,35 interface INFO:
	 stdout 2017-09-01T20:26:28.031724:  Iteration 14 (of 50).   Current convergence value = 0.000227609 (threshold = 0)
170901-20:26:28,537 interface INFO:
	 stdout 2017-09-01T20:26:28.537597:  Iteration 15 (of 50).   Current convergence value = 0.000223435 (threshold = 0)
170901-20:26:28,538 interface INFO:
	 stdout 2017-09-01T20:26:28.537597:  Iteration 16 (of 50).   Current convergence value = 0.000219261 (threshold = 0)
170901-20:26:28,539 interface INFO:
	 stdout 2017-09-01T20:26:28.537597:  Iterati

170901-20:26:34,594 interface INFO:
	 stdout 2017-09-01T20:26:34.593001:  Iteration 14 (of 50).   Current convergence value = 0.000744105 (threshold = 0)
170901-20:26:34,595 interface INFO:
	 stdout 2017-09-01T20:26:34.593001:  Iteration 15 (of 50).   Current convergence value = 0.000727847 (threshold = 0)
170901-20:26:35,97 interface INFO:
	 stdout 2017-09-01T20:26:35.097665:  Iteration 16 (of 50).   Current convergence value = 0.000717031 (threshold = 0)
170901-20:26:35,98 interface INFO:
	 stdout 2017-09-01T20:26:35.097665:  Iteration 17 (of 50).   Current convergence value = 0.000695444 (threshold = 0)
170901-20:26:35,99 interface INFO:
	 stdout 2017-09-01T20:26:35.097665:  Iteration 18 (of 50).   Current convergence value = 0.000681086 (threshold = 0)
170901-20:26:35,100 interface INFO:
	 stdout 2017-09-01T20:26:35.097665:  Iteration 19 (of 50).   Current convergence value = 0.000667994 (threshold = 0)
170901-20:26:35,601 interface INFO:
	 stdout 2017-09-01T20:26:35.601784:  Itera

170901-20:26:41,661 interface INFO:
	 stdout 2017-09-01T20:26:41.660603:  Iteration 17 (of 50).   Current convergence value = 0.00123145 (threshold = 0)
170901-20:26:41,667 interface INFO:
	 stdout 2017-09-01T20:26:41.660603:  Iteration 18 (of 50).   Current convergence value = 0.00121491 (threshold = 0)
170901-20:26:41,669 interface INFO:
	 stdout 2017-09-01T20:26:41.660603:  Iteration 19 (of 50).   Current convergence value = 0.00119798 (threshold = 0)
170901-20:26:42,171 interface INFO:
	 stdout 2017-09-01T20:26:42.171354:  Iteration 20 (of 50).   Current convergence value = 0.00118369 (threshold = 0)
170901-20:26:42,172 interface INFO:
	 stdout 2017-09-01T20:26:42.171354:  Iteration 21 (of 50).   Current convergence value = 0.00117709 (threshold = 0)
170901-20:26:42,173 interface INFO:
	 stdout 2017-09-01T20:26:42.171354:  Iteration 22 (of 50).   Current convergence value = 0.00116906 (threshold = 0)
170901-20:26:42,676 interface INFO:
	 stdout 2017-09-01T20:26:42.675884:  Iteratio

170901-20:26:46,758 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:       Debug: Off
170901-20:26:46,759 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:       Object Name: 
170901-20:26:46,760 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:       Observers: 
170901-20:26:46,763 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:         none
170901-20:26:46,764 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:       Thread Count: 1
170901-20:26:46,765 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:       Global Maximum Number Of Threads: 128
170901-20:26:46,766 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:       Global Default Number Of Threads: 1
170901-20:26:46,768 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:     CoordinateTolerance: 1e-06
170901-20:26:46,769 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:     DirectionTolerance: 1e-06
170901-20:26:46,770 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:     Mask label: 1
170901-20:26:46

170901-20:26:46,861 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:             Pointer: 0x213bd80
170901-20:26:46,862 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:             Container manages memory: true
170901-20:26:46,864 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:             Size: 1331
170901-20:26:46,865 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:             Capacity: 1331
170901-20:26:46,866 interface INFO:
	 stdout 2017-09-01T20:26:46.721779:Elapsed time: 26.4701
Renaming filename: sub-01_ses-retest_T1w_brain_pve_0.nii.gz
New filename: sub-01_ses-retest_T1w_brain_class-CSF_probtissue.nii.gz
Renaming filename: sub-01_ses-retest_T1w_brain_pveseg.nii.gz
New filename: sub-01_ses-retest_T1w_brain_pveseg.nii.gz
Renaming filename: sub-01_ses-retest_T1w_brain_pve_2.nii.gz
New filename: sub-01_ses-retest_T1w_brain_class-WM_probtissue.nii.gz
Renaming filename: sub-01_ses-retest_T1w_brain_mixeltype.nii.gz
New filename: sub-01_ses-retest_T1w_brain_mixeltyp